In [1]:
""""
Send video, frame-by-frame to Kafka interface
- Frame is encoded into ascii so no one gets upset with the data.
- Frame will be decomposed into chunks of 'CHUNK_SIZE'. When debugging found Kafka would not send message if it went over threshold.
- Receiving test notebook VideoRcvKafka
- The Steams application VideoRcvKafka recieves the encode image and scores it with Model.

"""
import kafka
import os
import sys
import json
import base64
import ssl
import time
import datetime
import io
from PIL import Image
import logging
import cv2
import matplotlib.pyplot as plt
import numpy as np
if '../juypter' not in sys.path:
    sys.path.insert(0, '../juypter')
import credential
import streams_aid as aid

logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))
# img_encoded = str(base64.b64encode(response.content).decode("utf-8"))
# img_encoded = str(base64.b64encode(img).decode('utf-8'))


def bts_to_img(bts):
    buff = np.fromstring(bts, np.uint8)
    buff = buff.reshape(1, -1)
    img = cv2.imdecode(buff, cv2.IMREAD_COLOR)
    return img


def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


def encode_img(img):
    """must be easier way"""
    with io.BytesIO() as output:
        img.save(output, format="JPEG")
        contents = output.getvalue()
    return base64.b64encode(contents).decode('ascii')


def decode_img(bin64):
    """must be easier way"""
    img = Image.open(io.BytesIO(base64.b64decode(bin64)))
    return img


CHUNK_SIZE = 100000         # maximum number of bytes to transmit at a time

def video_kafka(video_url, kafka_prod, kafka_topic='VideoFrame', frame_modulo=24, send_wait=.25, debug=False):
    """Send video via Kafka

    :param video_url: url of video to pull in and send
    :param kafka_prod: the handle to sent out messages on kafka
    :param frame_modulo: send every x frames
    :param send_wait: after sending a frame wait time
    :param debug: decode image and write out to verify
    :return: None

    """
    frame_num = 0

    cap = cv2.VideoCapture(video_url)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret is False:
            break
        frame_num += 1
        if not(frame_num % frame_modulo):
            image_encoded =encode_img(Image.fromarray(frame, 'RGB'))
            if debug:
                # debugging - render what we will send.
                img_raw = decode_img(image_encoded)
                plt.imshow(img_raw)
                plt.show()
            # break down frame into chunks
            chunks = [image_encoded[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE] for i in
                      range((len(image_encoded) + CHUNK_SIZE - 1) // CHUNK_SIZE)]
            # send the chunks.
            for idx, chunk in enumerate(chunks):
                logging.debug("chunking - {}  #chunks :{} idx:{} len(chunk):{}".format(video_url, len(chunks), idx, len(chunk)))
                chunk_content = {'video': video_url,
                       'frame': frame_num,
                       'chunk_idx':idx,
                       'chunk_total':len(chunks),
                       'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
                       'data': chunk
                }
                kafka_prod.send(kafka_topic, value=json.dumps(chunk_content).encode('utf-8'))
            ## finish the frame frame
            chunk_complete = {'video': video_url,
                   'frame': frame_num,
                    'chunk_idx': len(chunks),
                    'chunk_total': len(chunks),
                   'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
                   'data': ""
                   }
            logging.info("Transmit frame #{}".format(chunk_content["frame"]))
            kafka_prod.send(kafka_topic, value=json.dumps(chunk_complete).encode('utf-8'))
            time.sleep(send_wait)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return

def kafka_producer(credentials):
    """
    Open the connection to the kafka producer
    :param credentials:
    :return: kafka producer

    Request is responsilbe for closing producer.
    """
    prod = None
    while prod is None:
        try:
            prod = kafka.KafkaProducer(bootstrap_servers=credentials["kafka_brokers_sasl"],
                                       security_protocol="SASL_SSL",
                                       sasl_mechanism="PLAIN",
                                       sasl_plain_username=credentials["user"],
                                       sasl_plain_password=credentials["api_key"],
                                       ssl_cafile=ssl.get_default_verify_paths().cafile)

        except kafka.errors.NoBrokersAvailable:
            logging.warning("No Brokers Available. Retrying ...")
            time.sleep(1)
            prod = None
    return prod

In [ ]:
"""
Selection of video use - 
 - Internet archive has many more.
"""

TOPIC="VideoFrame"

@aid.catchInterrupt
def videoStream(topic="VideoFrame", videoUrl="https://archive.org/download/popeye_shuteye_popeye/popeye_shuteye_popeye_512kb.mp4"):
    creds = json.loads(credential.magsEventStream)
    prod = kafka_producer(creds,)
    video_kafka(videoUrl, prod,  kafka_topic=topic, send_wait=1, frame_modulo=24)
    prod.close()
    
video_urls = [
'https://ia803104.us.archive.org/21/items/nwctc-Crystal_Airport_Changes_to_Move_Forward/Crystal_Airport_Changes_to_Move_Forward.mp4',
'https://ia801309.us.archive.org/12/items/ISS-Tours/ISS-Tour_Flight-day-highlights_DI_2015_007_1256_220882.mp4',
'https://ia802705.us.archive.org/14/items/KarlFriedrichDraisTheBicycle/Karl%20Friedrich%20Drais%20The%20Bicycle.mp4',
'https://ia800504.us.archive.org/30/items/doa_1949/doa_1949_512kb.mp4',
'https://archive.org/download/popeye_shuteye_popeye/popeye_shuteye_popeye_512kb.mp4',
'https://archive.org/download/MuhammadAliVsSonnyListon/MuhammadAliVsSonnyListon_512kb.mp4',
'https://archive.org/download/bb_snow_white/bb_snow_white_512kb.mp4',
]

videoStream(videoUrl=video_urls[3])

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-4 host=broker-3-qh451l9pr7s10q5v.kafka.svc01.us-south.eventstreams.cloud.ibm.com:9093 <connecting> [IPv4 ('52.116.39.5', 9093)]>: connecting to broker-3-qh451l9pr7s10q5v.kafka.svc01.us-south.eventstreams.cloud.ibm.com:9093 [('52.116.39.5', 9093) IPv4]
INFO:kafka.conn:Probing node bootstrap-4 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-4 host=broker-3-qh451l9pr7s10q5v.kafka.svc01.us-south.eventstreams.cloud.ibm.com:9093 <handshake> [IPv4 ('52.116.39.5', 9093)]>: Loading SSL CA from /Users/siegenth/opt/anaconda3/envs/py36/ssl/cert.pem
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-4 host=broker-3-qh451l9pr7s10q5v.kafka.svc01.us-south.eventstreams.cloud.ibm.com:9093 <authenticating> [IPv4 ('52.116.39.5', 9093)]>: Authenticated as token via PLAIN
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-4 host=broker-3-qh451l9pr7s10q5v.kafka.svc01.us-south.eventstreams.cloud.ibm.com:9093 <authenticating> [IPv4 ('52.1

INFO:root:Transmit frame #4584
INFO:root:Transmit frame #4608
INFO:root:Transmit frame #4632
INFO:root:Transmit frame #4656
INFO:root:Transmit frame #4680
INFO:root:Transmit frame #4704
INFO:root:Transmit frame #4728
INFO:root:Transmit frame #4752
INFO:root:Transmit frame #4776
INFO:root:Transmit frame #4800
INFO:root:Transmit frame #4824
INFO:root:Transmit frame #4848
INFO:root:Transmit frame #4872
INFO:root:Transmit frame #4896
INFO:root:Transmit frame #4920
INFO:root:Transmit frame #4944
INFO:root:Transmit frame #4968
INFO:root:Transmit frame #4992
INFO:root:Transmit frame #5016
INFO:root:Transmit frame #5040
INFO:root:Transmit frame #5064
INFO:root:Transmit frame #5088
INFO:root:Transmit frame #5112
INFO:root:Transmit frame #5136
INFO:root:Transmit frame #5160
INFO:root:Transmit frame #5184
INFO:root:Transmit frame #5208
INFO:root:Transmit frame #5232
INFO:root:Transmit frame #5256
INFO:root:Transmit frame #5280
INFO:root:Transmit frame #5304
INFO:root:Transmit frame #5328
INFO:roo

INFO:root:Transmit frame #10920
INFO:root:Transmit frame #10944
INFO:root:Transmit frame #10968
INFO:root:Transmit frame #10992
INFO:root:Transmit frame #11016
INFO:root:Transmit frame #11040
INFO:root:Transmit frame #11064
INFO:root:Transmit frame #11088
INFO:root:Transmit frame #11112
INFO:root:Transmit frame #11136
INFO:root:Transmit frame #11160
INFO:root:Transmit frame #11184
INFO:root:Transmit frame #11208
INFO:root:Transmit frame #11232
INFO:root:Transmit frame #11256
INFO:root:Transmit frame #11280
INFO:root:Transmit frame #11304
INFO:root:Transmit frame #11328
INFO:root:Transmit frame #11352
INFO:root:Transmit frame #11376
INFO:root:Transmit frame #11400
INFO:root:Transmit frame #11424
INFO:root:Transmit frame #11448
INFO:root:Transmit frame #11472
INFO:root:Transmit frame #11496
INFO:root:Transmit frame #11520
INFO:root:Transmit frame #11544
INFO:root:Transmit frame #11568
INFO:root:Transmit frame #11592
INFO:root:Transmit frame #11616
INFO:root:Transmit frame #11640
INFO:roo

INFO:root:Transmit frame #17088
INFO:root:Transmit frame #17112
INFO:root:Transmit frame #17136
INFO:root:Transmit frame #17160
INFO:root:Transmit frame #17184
INFO:root:Transmit frame #17208
INFO:root:Transmit frame #17232
INFO:root:Transmit frame #17256
INFO:root:Transmit frame #17280
INFO:root:Transmit frame #17304
INFO:root:Transmit frame #17328
INFO:root:Transmit frame #17352
INFO:root:Transmit frame #17376
INFO:root:Transmit frame #17400
INFO:root:Transmit frame #17424
INFO:root:Transmit frame #17448
INFO:root:Transmit frame #17472
INFO:root:Transmit frame #17496
INFO:root:Transmit frame #17520
INFO:root:Transmit frame #17544
INFO:root:Transmit frame #17568
INFO:root:Transmit frame #17592
INFO:root:Transmit frame #17616
INFO:root:Transmit frame #17640
INFO:root:Transmit frame #17664
INFO:root:Transmit frame #17688
INFO:root:Transmit frame #17712
INFO:root:Transmit frame #17736
INFO:root:Transmit frame #17760
INFO:root:Transmit frame #17784
INFO:root:Transmit frame #17808
INFO:roo